# Create Garment dicts

In [20]:
from utils.io import pickle_load, pickle_dump

In [21]:
garment_dict_path = '/local/home/agrigorev/Data/02_Projects/hood_data/aux_data/garments_dict.pkl'
garment_dict_path = '/local/home/agrigorev/Data/02_Projects/ccraft_data/aux_data/train_wdetailed2.pkl'

garments_dict = pickle_load(garment_dict_path)

In [22]:
out_dir = "/local/home/agrigorev/Data/02_Projects/ccraft_data/aux_data/garment_dicts"

for k, v in garments_dict.items():
    v['name'] = k
    pickle_dump(v, f"{out_dir}/{k}.pkl")

In [1]:
%load_ext autoreload
%autoreload 2

# Test inference with individual garment dicts

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

HOOD_PROJECT =  "/local/home/agrigorev/Workdir/00_Projects/contourcraft_private"
HOOD_DATA = "/local/home/agrigorev/Data/02_Projects/ccraft_data"

os.environ["HOOD_PROJECT"] = HOOD_PROJECT
os.environ["HOOD_DATA"] = HOOD_DATA

In [3]:
from utils.validation import apply_material_params
from utils.validation import load_runner_from_checkpoint
from utils.arguments import load_params
from utils.common import move2device
from utils.io import pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path


# Set material paramenters, see configs/cvpr.yaml for the training ranges for each parameter
material_dict = dict()
material_dict['density'] = 0.20022
material_dict['lame_mu'] = 23600.0
material_dict['lame_lambda'] = 44400
material_dict['bending_coeff'] = 3.962e-05


# ====================================================================================================

models_dir = Path(DEFAULTS.data_root) / 'trained_models'

# Choose the model and the configuration file
config_name = 'ccraft_indgd'
checkpoint_path = models_dir / 'contourcraft.pth'


# ====================================================================================================


# load the config from .yaml file and load .py modules specified there
modules, experiment_config = load_params(config_name)

# modify the config to use it in validation 
experiment_config = apply_material_params(experiment_config, material_dict)

# load Runner object and the .py module it is declared in
runner_module, runner = load_runner_from_checkpoint(checkpoint_path, modules, experiment_config)


/local/home/agrigorev/miniforge3/envs/ccraft/lib/python3.10/site-packages/torch_cluster/nearest.py:3: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.1)
  import scipy.cluster


Warp 1.5.1 initialized:
   CUDA Toolkit 12.6, Driver 12.7
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 3060" (12 GiB, sm_86, mempool enabled)
   Kernel cache:
     /local/home/agrigorev/.cache/warp/1.5.1


/local/home/agrigorev/Workdir/00_Projects/contourcraft_private/utils/validation.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(checkpoint_path)


In [9]:
# file with the pose sequence
from utils.validation import create_postcvpr_one_sequence_dataloader

# If True, the SMPL(-X) poses are slightly modified to avoid hand-body self-penetrations. The technique is adopted from the code of SNUG 
separate_arms = True

# path to the pose sequence file
# sequence_path =  '/local/home/agrigorev/Data/00_Datasets/AMASS/smplx/CMU//01/01_01_stageii.npz'
sequence_path =  '/local/home/agrigorev/Data/00_Datasets/AMASS/smpl/CMU//01/01_01_poses.npz'

# name of the garment to simulate
# garment_name = 'cindy_020_combined_test'
garment_name = 'ben_004::top_coat,ben_004::scarf'

# It can be a comma-separated list of individual garments
# garment_name = 'cindy_020::bottom_skirt, cindy_020::top_blouse'

garment_dict_file = 'garments_dict.pkl'


# gender of the body model, sould be the same as the one used to create the garment
# gender = 'female'
gender = 'male'


# ====================================================================================================

# Choose the type of the pose sequence you want to use: 'cmu_npz_smpl', 'cmu_npz_smplx', 'hood_pkl'

# to use AMASS SMPL-X pose sequence
# sequence_loader = 'cmu_npz_smplx'

# to use AMASS SMPL pose sequence
sequence_loader = 'cmu_npz_smpl'

# to use our (legacy) SMPL pose sequence
# sequence_loader = 'hood_pkl'


# ====================================================================================================


dataloader = create_postcvpr_one_sequence_dataloader(sequence_path, garment_name, 
                                            garment_dict_file, sequence_loader=sequence_loader, 
                                            obstacle_dict_file=None, config="ccraft_indgd", gender=gender)

{'data_root': '/local/home/agrigorev/Data/00_Datasets/AMASS/smpl/CMU//01', 'single_sequence_file': '01_01_poses', 'single_sequence_garment': 'ben_004::top_coat,ben_004::scarf', 'sequence_loader': 'cmu_npz_smpl', 'obstacle_dict_file': None, 'gender': 'male'}


In [11]:
sequence = next(iter(dataloader))
sequence = move2device(sequence, 'cuda:0')
trajectories_dict = runner.valid_rollout(sequence,  bare=True, n_steps=20)

100%|██████████| 20/20 [00:04<00:00,  4.58it/s]


In [12]:
from utils.io import pickle_dump

out_path = '/local/home/agrigorev/Data/temp/out.pkl'
pickle_dump(trajectories_dict, out_path)

# Check datalists

In [8]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict

def proc_id(old_id):
    parts = old_id.split('_')

    person_id = parts[-2]
    seq_id = parts[-1]

    relative_path = f"{person_id}/{person_id}_{seq_id}_poses.npz"

    return relative_path

def get_lengths(amass_root, id_list):
    ids_unique = list(set(id_list))
    lengths = {}

    for id in ids_unique:
        path = f"{amass_root}/{id}"

        if not os.path.exists(path):
            print(f"Path {id} does not exist")
            lengths[id] = -1
            continue

        data = dict(np.load(path, allow_pickle=True))
        l = data['poses'].shape[0]
        lengths[id] = l     

    return lengths   

datasplit_path = '/local/home/agrigorev/Data/02_Projects/ccraft_data/aux_data/datasplits/train_multig_novest.csv'
datasplit = pd.read_csv(datasplit_path)

datasplit_new = pd.DataFrame()

In [9]:
ids_old = datasplit['id'].values
ids_new = [proc_id(old_id) for old_id in ids_old]

amass_root = '/local/home/agrigorev/Data/00_Datasets/AMASS/smpl/CMU'
length_dict = get_lengths(amass_root, ids_new)

new_df_dict = defaultdict(list)

for i, id in enumerate(ids_new):
    l = length_dict[id]

    if l > 0:
        new_df_dict['id'].append(id)
        new_df_dict['length'].append(length_dict[id])
        new_df_dict['garment'].append(datasplit['garment'].values[i])
        new_df_dict['gender'].append(datasplit['gender'].values[i])

datasplit_new = pd.DataFrame()

for k, v in new_df_dict.items():
    datasplit_new[k] = v

Path 128/128_07_poses.npz does not exist
Path 104/104_04_poses.npz does not exist
Path 73/73_09_poses.npz does not exist
Path 104/104_53_poses.npz does not exist
Path 76/76_05_poses.npz does not exist
Path 144/144_30_poses.npz does not exist
Path 128/128_04_poses.npz does not exist
Path 128/128_02_poses.npz does not exist
Path 128/128_05_poses.npz does not exist
Path 104/104_17_poses.npz does not exist
Path 128/128_03_poses.npz does not exist
Path 104/104_11_poses.npz does not exist
Path 128/128_06_poses.npz does not exist
Path 26/26_11_poses.npz does not exist
Path 76/76_11_poses.npz does not exist


In [10]:
datasplit_new.to_csv('/local/home/agrigorev/Data/02_Projects/ccraft_data/aux_data/datasplits/train_ccraft.csv', index=False)

# train debug

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import torch

from utils.arguments import load_params, create_modules

/local/home/agrigorev/miniforge3/envs/ccraft/lib/python3.10/site-packages/torch_cluster/nearest.py:3: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.1)
  import scipy.cluster


In [3]:
modules, config = load_params(config_name='ccraft_train_s1')
dataloader_ms, runner_module, runner, aux_modules = create_modules(modules, config)

dataloaders_dict = dict()
for dataloader_name, dataloader in dataloader_ms.items():
    dataloaders_dict[dataloader_name] = dataloader.create_dataloader()

# dataloader_inference = dataloaders_dict['short']

Warp 1.5.1 initialized:
   CUDA Toolkit 12.6, Driver 12.7
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 3060" (12 GiB, sm_86, mempool enabled)
   Kernel cache:
     /local/home/agrigorev/.cache/warp/1.5.1


In [4]:
dataloader_short = dataloaders_dict['short']
dataloader_long = dataloaders_dict['long']

In [5]:
sample_short = next(iter(dataloader_short))
sample_long = next(iter(dataloader_long))

In [ ]:
global_step = runner_module.run_epoch(runner, aux_modules, dataloaders_dict, 0, config, None,
                                global_step=0)

ccraft_train_s1:   0%|          | 0/91514 [00:00<?, ?it/s]

Module utils.warp_u.proximity b91666f load on device 'cuda:0' took 0.75 ms  (cached)


ccraft_train_s1:   0%|          | 19/91514 [00:08<11:34:53,  2.19it/s]


KeyboardInterrupt: 

: 

# Make debug datalists for new and old

In [ ]:
import pandas as pd

old_df_path = '/local/home/agrigorev/Data/02_Projects/ccraft_data/aux_data/datasplits/train_multig_novest.csv'
old_datasplit = pd.read_csv(old_df_path)
old_datasplit_id = 


new_df_path = '/local/home/agrigorev/Data/02_Projects/ccraft_data/aux_data/datasplits/train_ccraft.csv'
new_datasplit = pd.read_csv(new_df_path)

In [2]:
old_datasplit

,Unnamed: 0.1,Unnamed: 0,id,length,garment,gender
0,0,0.0,tshirt_shape14_104_11,1320,tshirt,female
1,1,1.0,tshirt_shape08_128_02,390,tshirt,female
2,2,2.0,tshirt_shape14_128_03,390,tshirt,female
3,3,3.0,tshirt_shape03_91_61,880,tshirt,female
4,4,4.0,tshirt_shape10_46_01,1520,tshirt,female
...,...,...,...,...,...,...
29167,32703,NaN,tshirt_shape09_144_26,1480,"ben_004::top_coat,ben_004::scarf",male
29168,32704,NaN,tshirt_shape04_91_38,1120,"ben_004::top_coat,ben_004::scarf",male
29169,32705,NaN,tshirt_shape16_104_11,1320,"ben_004::top_coat,ben_004::scarf",male
29170,32706,NaN,tshirt_shape03_111_37,790,"ben_004::top_coat,ben_004::scarf",male


In [3]:
new_datasplit

,id,length,garment,gender
0,91/91_61_poses.npz,362,tshirt,female
1,46/46_01_poses.npz,616,tshirt,female
2,111/111_37_poses.npz,324,tshirt,female
3,108/108_12_poses.npz,137,tshirt,female
4,144/144_28_poses.npz,2286,tshirt,female
...,...,...,...,...
20752,111/111_05_poses.npz,926,"ben_004::top_coat,ben_004::scarf",male
20753,144/144_26_poses.npz,3347,"ben_004::top_coat,ben_004::scarf",male
20754,91/91_38_poses.npz,458,"ben_004::top_coat,ben_004::scarf",male
20755,111/111_37_poses.npz,324,"ben_004::top_coat,ben_004::scarf",male


# Pretrain

### config
```python
self.mcfg.no_world_edges_every = 2
self.mcfg.short_is_training = True
self.mcfg.long_is_training = True
self.mcfg.nocollect_after = 2
```


### forward_short
```python
use_wedges_seq = True
if self.mcfg.no_world_edges_every > 0 and self.short_steps % self.mcfg.no_world_edges_every == 0:
    use_wedges_seq = False

for i in range(roll_steps):
    is_safecheck = self.safecheck

    use_wedges = (roll_steps == 1) or (i > 0)
    use_wedges = use_wedges and use_wedges_seq

    if not use_wedges:
        is_safecheck = False

    if self.mcfg.nocollect_after > 0:
        is_training = self.mcfg.short_is_training and (i < self.mcfg.nocollect_after)
    else:
        is_training = self.mcfg.short_is_training
    if i == 0 and roll_steps > 1:
        is_training = False

    sample_step = self.model(sample_step, world_edges=use_wedges, is_training=is_training, fake_icontour=True)

```

### forward_long
```python

use_wedges_seq = True
if self.mcfg.no_world_edges_every > 0 and self.short_steps % self.mcfg.no_world_edges_every == 0:
    use_wedges_seq = False

for i in range(roll_steps):
    is_safecheck = self.safecheck

    use_wedges = True
    use_wedges = use_wedges and use_wedges_seq
    if not use_wedges:
        is_safecheck = False


    if self.mcfg.nocollect_after > 0:
        is_training = self.mcfg.long_is_training and (i < self.mcfg.nocollect_after)
    else:
        is_training = self.mcfg.long_is_training
    sample_step = self.model(sample_step, world_edges=use_wedges, is_training=is_training, fake_icontour=True)
```

# Stage 2

### config
```python
self.mcfg.no_world_edges_every = 2
self.mcfg.nocollect_after = 2
self.mcfg.short_is_training = True
```


### forward_short
```python
use_wedges_seq = True
if self.mcfg.no_world_edges_every > 0 and self.short_steps % self.mcfg.no_world_edges_every == 0:
    use_wedges_seq = False


for i in range(roll_steps):
    is_safecheck = False

    use_wedges = (roll_steps == 1) or (i > 0)
    use_wedges = use_wedges and use_wedges_seq

    if self.mcfg.nocollect_after > 0:
        is_training = self.mcfg.short_is_training and (i < self.mcfg.nocollect_after)
    else:
        is_training = self.mcfg.short_is_training
    if i == 0 and roll_steps > 1:
        is_training = False

    fake_icontour = not use_wedges
    sample_step = self.model(sample_step, world_edges=use_wedges, is_training=is_training, fake_icontour=fake_icontour)
```



### forward_long
```python
use_wedges_seq = True
if self.mcfg.no_world_edges_every > 0 and self.short_steps % self.mcfg.no_world_edges_every == 0:
    use_wedges_seq = False

for i in range(roll_steps):
    is_safecheck = False

    use_wedges = True
    use_wedges = use_wedges and use_wedges_seq
    if not use_wedges:
        is_safecheck = False

    if self.mcfg.nocollect_after > 0:
        is_training = self.mcfg.long_is_training and (i < self.mcfg.nocollect_after)
    else:
        is_training = self.mcfg.long_is_training

    fake_icontour = not use_wedges

    sample_step = self.model(sample_step, world_edges=use_wedges, is_training=is_training, fake_icontour=fake_icontour)
```